# 01 · Analista conversacional
_UI mínima tipo chat orquestada por ChatRunner (no modifica agents/ ni tools/)._

In [1]:

# ==== Boot / Init ====
from pathlib import Path
import os, sys

# Localiza raíz del proyecto
CANDIDATES = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
ROOT = None
for c in CANDIDATES:
    if (c / "config" / "settings.py").exists():
        ROOT = c; break
if ROOT is None:
    raise FileNotFoundError("No encuentro 'config/settings.py'.")

# Asegura paquetes y cwd
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
for d in ["agents","tools","config"]:
    (ROOT/d/"__init__.py").touch(exist_ok=True)

# Carga settings y entorno Vertex/Google (sin romper si falla algo menor)
from config.settings import PROJECT_ID, REGION, BQ_LOCATION, TABLA_BASE_FQN
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_PROJECT"]      = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"]     = REGION

print("ROOT:", ROOT)
print("Proyecto:", PROJECT_ID, "| Región:", REGION, "| BQ:", BQ_LOCATION)
print("Tabla base:", TABLA_BASE_FQN)

# Ping BigQuery opcional (silenciado si falla)
try:
    from google.cloud import bigquery
    bq = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
    print("Ping BQ:", list(bq.query("SELECT 1 AS ok").result()))
except Exception as e:
    print("Aviso: BigQuery ping omitido →", e)

print("Entorno listo.")


ROOT: /home/jupyter/sgr_gitlab/proyecto_bbdd01_ia/BBDD01_V_AGENTES/project
Proyecto: go-cxb-bcx-data9-dtwsgrp01 | Región: europe-southwest1 | BQ: europe-southwest1
Tabla base: `go-cxb-bcx-data9-dtwsgrp01.dtwsgr_ds01.BBDD_01_LIGHT`
Ping BQ: [Row((1,), {'ok': 0})]
Entorno listo.


In [2]:

from importlib import reload
import agents.loop_runner as lr
reload(lr)

# Ajusta aquí el modelo/temperatura si quieres
runner = lr.ChatRunner(model="gemini-2.5-pro", temperature=0.2)
print("Runner listo:", runner)


Umbral BigQuery activo: 10240 MB
Runner listo: <agents.loop_runner.ChatRunner object at 0x7ca78a9d1840>


In [3]:

import ipywidgets as w
import pandas as pd
from IPython.display import display, Markdown

# Controles
q_input   = w.Textarea(placeholder="Escribe tu pregunta…", layout=w.Layout(width="100%", height="80px"))
prefer_web= w.Checkbox(value=False, description="Preferir web")
web_only  = w.Checkbox(value=False, description="Solo web")
show_df   = w.Checkbox(value=True,  description="Mostrar tabla (si hay)")
show_sql  = w.Checkbox(value=False, description="Mostrar SQL (si hay)")

send_btn  = w.Button(description="Enviar", button_style="primary")
clear_btn = w.Button(description="Limpiar historial")

sql_area  = w.Textarea(disabled=True, layout=w.Layout(width="100%", height="120px"))
sql_acc   = w.Accordion(children=[sql_area], selected_index=None)
sql_acc.set_title(0, "SQL generado")

out_area  = w.Output()
history   = w.HTML(value="")

controls  = w.HBox([prefer_web, web_only, show_df, show_sql, send_btn, clear_btn])
display(w.VBox([q_input, controls, sql_acc, out_area, history]))

def _fmt_sources(web_ctx: dict) -> str:
    try:
        srcs = (web_ctx or {}).get("sources") or []
        if not srcs:
            return ""
        items = []
        for s in srcs:
            url   = s.get("url") or ""
            title = s.get("title") or url
            if url:
                items.append(f"- [{title}]({url})")
        return "\n\n**Fuentes:**\n" + "\n".join(items)
    except Exception:
        return ""

def _preview_df(df_like, max_rows=10):
    if df_like is None:
        return
    try:
        if isinstance(df_like, pd.DataFrame):
            df = df_like
        else:
            df = pd.DataFrame(df_like)
        display(df.head(max_rows))
    except Exception as e:
        print("No se pudo mostrar la tabla:", e)

def _render_answer(res: dict):
    out_area.clear_output()
    with out_area:
        text = res.get("text") if isinstance(res, dict) else str(res)
        web  = res.get("web")  if isinstance(res, dict) else None
        df   = res.get("df")   if isinstance(res, dict) else None
        sql  = res.get("sql")  if isinstance(res, dict) else ""

        display(Markdown(text if isinstance(text, str) else str(text)))

        src_md = _fmt_sources(web if isinstance(web, dict) else {})
        if src_md:
            display(Markdown(src_md))

        if show_df.value and df is not None:
            _preview_df(df, max_rows=10)

        sql_area.value = sql or ""
        sql_acc.selected_index = 0 if (show_sql.value and sql_area.value) else None

def _append_hist(user_q: str, answer_txt: str):
    snippet = (answer_txt or "")[:300].replace("\n", " ")
    history.value = (history.value or "") + f"<hr><b>Tú:</b> {user_q}<br><b>Asistente:</b> {snippet}…"

def _on_send(_):
    q = (q_input.value or "").strip()
    if not q:
        return
    try:
        res = runner.answer(q, prefer_web=prefer_web.value, web_only=web_only.value)
        _render_answer(res)
        _append_hist(q, res.get("text","") if isinstance(res, dict) else str(res))
    except Exception as e:
        _render_answer({"text": f"⚠️ Error: {e}", "notes": []})

def _on_clear(_):
    history.value = ""
    out_area.clear_output()
    sql_area.value = ""

send_btn.on_click(_on_send)
clear_btn.on_click(_on_clear)

print("UI lista. Escribe tu pregunta y pulsa Enviar.")


UI lista. Escribe tu pregunta y pulsa Enviar.


In [4]:

print("T1 · Web-only (OPA BBVA/Sabadell)")
try:
    r = runner.answer("Últimas noticias de la OPA de BBVA sobre Banco Sabadell: resumen y 3-5 fuentes.", web_only=True)
    print((r.get("text") or "")[:400], "…")
except Exception as e:
    print("T1 ERROR:", e)

print("\nT2 · SQL (evolución 2025; con fallback si excede umbral activo)")
try:
    r = runner.answer("Evolución del TOTAL_RIESGO por MES en 2025 (línea).", prefer_web=False)
    print((r.get("text") or "")[:400], "…")
    print("Notas:", r.get("notes"))
except Exception as e:
    print("T2 ERROR:", e)


T1 · Web-only (OPA BBVA/Sabadell)
[audit_log] Error silenciado: module 'google.cloud.bigquery._helpers' has no attribute 'utcnow'
### Últimas noticias de la OPA de BBVA sobre Banco Sabadell: resumen y 3-5 fuentes.
**Contexto externo (resumen):**
El BBVA fracasa en su opa sobre el Sabadell al lograr solo un 25% de aceptación del capital. El presidente del BBVA, Carlos Torres, ha agradecido en un vídeo a los accionistas del Banco Sabadell que han mostrado su apoyo al proyecto de unión. El evento tendrá lugar el 15 de noviembre …

T2 · SQL (evolución 2025; con fallback si excede umbral activo)
[audit_log] Error silenciado: module 'google.cloud.bigquery._helpers' has no attribute 'utcnow'
### Evolución del TOTAL_RIESGO por MES en 2025 (línea)
- Filas: 9
- Rango MES: 202501–202509
**Notas:** Auditoría de datos omitida: too many values to unpack (expected 2) | Visualización omitida: 'Plan' object has no attribute 'empty'

_Unidades: millones de euros (M€)._ …
Notas: ['Auditoría de datos omit